# Karim Kuserbaev

## Funcs

In [1]:
def get_df_low(code: int):
    train_df = pd.read_parquet(f"../data/interim/train_data_lm_{code}.pqt")
    test_df = pd.read_parquet(f"../data/interim/test_data_lm_{code}.pqt")

    cat_cols = [
        "channel_code", "city", "city_type",
        "okved", "segment", "start_cluster",
        "index_city_code", "ogrn_month", "ogrn_year",
    ]

    train_df[cat_cols] = train_df[cat_cols].astype("category")
    test_df[cat_cols] = test_df[cat_cols].astype("category")

    df = pd.concat([train_df, test_df], ignore_index=True)
    return df



def get_feature_description():
  feature_description = pd.read_excel("../data/feature_description.xlsx") # import feature_description to merge description
  feature_description.columns =  ['col_name', 'description', 'type']
  return feature_description



def cat_column_summary(df):
    """Для категорий всех столбцов"""
    df = df.select_dtypes(exclude="number")
    summary_data = []

    for col_name in df.columns:
        col_dtype = df[col_name].dtype
        num_of_nulls = df[col_name].isnull().sum()
        num_of_non_nulls = df[col_name].notnull().sum()
        num_of_distinct_values = df[col_name].nunique()

        if num_of_distinct_values <= 10:
            distinct_values_counts = df[col_name].value_counts().to_dict()
        else:
            top_10_values_counts = df[col_name].value_counts().head(10).to_dict()
            distinct_values_counts = {k: v for k, v in sorted(top_10_values_counts.items(), key=lambda item: item[1], reverse=True)}

        summary_data.append({
            'col_name': col_name,
            'col_dtype': col_dtype,
            'num_of_nulls': num_of_nulls,
            'num_of_non_nulls': num_of_non_nulls,
            'num_of_distinct_values': num_of_distinct_values,
            'distinct_values_counts': distinct_values_counts
        })


    summary_df = pd.DataFrame(summary_data)
    summary_df_with_fd = pd.merge(summary_df, feature_description, on='col_name', suffixes=('_left', '_right'), how='left')

    return summary_df_with_fd

def num_column_summary(df):
    """Для численных всех столбцов"""

    df = df.select_dtypes(include="number")
    summary_data = []

    for col_name in df.columns:
        col_dtype = df[col_name].dtype
        num_of_nulls = df[col_name].isnull().sum()
        num_of_non_nulls = df[col_name].notnull().sum()
        num_of_distinct_values = df[col_name].nunique()
        mean = df[col_name].mean()
        std = df[col_name].std()
        min_val = df[col_name].min()
        q25 = df[col_name].quantile(0.25)
        q50 = df[col_name].quantile(0.50)
        q75 = df[col_name].quantile(0.75)
        max_val = df[col_name].max()


        if num_of_distinct_values <= 10:
            distinct_values_counts = df[col_name].value_counts().to_dict()
        else:
            top_10_values_counts = df[col_name].value_counts().head(10).to_dict()
            distinct_values_counts = {k: v for k, v in sorted(top_10_values_counts.items(), key=lambda item: item[1], reverse=True)}

        summary_data.append({
            'col_name': col_name,
            'col_dtype': col_dtype,
            'num_of_nulls': num_of_nulls,
            'num_of_non_nulls': num_of_non_nulls,
            'num_of_distinct_values': num_of_distinct_values,
            'mean': mean,
            'std': std,
            'min': min_val,
            'q25': q25,
            'q50': q50,
            'q75': q75,
            'max': max_val,
            'distinct_values_counts': distinct_values_counts

        })




    summary_df = pd.DataFrame(summary_data)
    summary_df_with_fd = pd.merge(summary_df, feature_description, on='col_name', suffixes=('_left', '_right'), how='left')

    return summary_df_with_fd
# Пустые значения


def get_nan_3_month_df(df):

  nan_3_month_df = pd.DataFrame(df['id'].unique(), columns=['id'])

  for column in df.columns[1:]:
    start_time = time.time()
    result = df.groupby('id')[column].apply(
        lambda x: x.isna().all()).reset_index()
    nan_3_month_df = pd.merge(
        nan_3_month_df, result, on='id', suffixes=('_left', '_right'), how='left')
    end_time = time.time()
  return nan_3_month_df



def process_nan(x):
    try:
        x = x.dropna(
        ).values[-1] if x.isna().any() and not x.isna().all() else np.nan
        return x
    except:
        print(x)
        return x




def get_nan_values_3_month_df(df):

  nan_3_month_df = pd.DataFrame(df['id'].unique(), columns=['id'])
  for column in df.columns[1:]:
    start_time = time.time()
    result = df.groupby('id')[column].apply(
        lambda x: process_nan(x)).reset_index()
    nan_3_month_df = pd.merge(
        nan_3_month_df, result, on='id', how='left')
    end_time = time.time()
    print(f"{column} {end_time - start_time}")
  return nan_3_month_df



def plt_null_df(df):

    plt.figure(figsize=(15, 8))

    plt.imshow(df, aspect="auto", interpolation="nearest", cmap="viridis")

    plt.xticks(range(len(df.columns)), df.columns, rotation='vertical')

    plt.xlabel("Column Number")
    plt.ylabel("Sample Number")

    plt.show()




# Корреляция
def plt_corr_df(df):

  corr_df = df.select_dtypes(include='number').corr().abs()

  plt.figure(figsize=(20, 15))
  dataplot = sns.heatmap(corr_df,
                        cmap="YlGnBu", annot=True, fmt=".1f",
                        annot_kws={'size': 5})
  plt.show()

def get_columns_corr(df, p: int):
    corr_df = df.select_dtypes(include='number').corr().abs()
    c1 = corr_df.unstack()

    c1_sorted = c1.sort_values(ascending=True)
    columns_above = {(col1, col2): c1_sorted[col1, col2] for col1, col2 in c1_sorted.index if c1_sorted[col1, col2] > p and col1 != col2}

    return dict(sorted(columns_above.items(), key=lambda item: item[1], reverse=True))

## Импорт библиотек и настройки

In [64]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [2]:
!pip install catboost

In [3]:
import numpy as np
import pandas as pd
import optuna

from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import os

from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier, Pool
import catboost
from catboost import *
from catboost import datasets


import numpy as np
import time
import json

import pandas as pd
pd.set_option('display.float_format', '{:.4f}'.format)
pd.set_option('display.max_rows', 93)

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

import warnings
from sklearn.utils import resample


# Отключить все предупреждения временно
warnings.filterwarnings("ignore")

# Ваш код, вызывающий предупреждение о переполнении

# Включить предупреждения обратно
# warnings.filterwarnings("default")


import matplotlib.pyplot as plt

from sklearn.ensemble import StackingClassifier, RandomForestClassifier, ExtraTreesClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB

In [4]:
!pip freeze | grep "numpy\|pandas\|lightgbm\|scikit-learn"

geopandas==0.14.3
lightgbm @ file:///tmp/lightgbm/lightgbm-4.2.0-py3-none-manylinux_2_31_x86_64.whl#sha256=26ed21477c12bb26edc4d6d51336cd43d5a8f7daf55ebbe27b0faf50ce96db23
numpy @ file:///home/conda/feedstock_root/build_artifacts/numpy_1707225380409/work/dist/numpy-1.26.4-cp310-cp310-linux_x86_64.whl#sha256=51131fd8fc130cd168aecaf1bc0ea85f92e8ffebf211772ceb16ac2e7f10d7ca
pandas==2.1.4
pandas-datareader==0.10.0
pandas-profiling==3.6.6
pandas-summary==0.2.0
pandasql==0.7.3
scikit-learn==1.2.2
scikit-learn-intelex==2024.2.0
sklearn-pandas==2.2.0


In [5]:
pd.options.display.max_rows = 93
pd.options.display.max_columns = 93

## Загрузка данных

In [116]:
df = pd.read_parquet("/kaggle/input/df-restore-cal-avg-start-cluster-3-pqt/df_restore_cal_avg_start_cluster_3.pqt")

df_org = pd.read_parquet("/kaggle/input/df-restore-cal-avg-start-cluster-3-pqt/df_0.89.pqt")
df_org1 = pd.read_parquet("/kaggle/input/df-restore-cal-avg-start-cluster-3-pqt/old_df_restore_cal_avg_start_cluster_3.pqt")



In [117]:
df_org1

,id,avg_a_oper_1m_month_1,avg_a_oper_1m_month_2,avg_a_oper_1m_month_3,avg_a_oper_3m_month_1,avg_a_oper_3m_month_2,avg_a_oper_3m_month_3,avg_b_oper_1m_month_1,avg_b_oper_1m_month_2,avg_b_oper_1m_month_3,avg_b_oper_3m_month_1,avg_b_oper_3m_month_2,avg_b_oper_3m_month_3,avg_c_oper_1m_month_1,avg_c_oper_1m_month_2,avg_c_oper_1m_month_3,avg_c_oper_3m_month_1,avg_c_oper_3m_month_2,avg_c_oper_3m_month_3,avg_cred_d_oper_1m_month_1,avg_cred_d_oper_1m_month_2,avg_cred_d_oper_1m_month_3,avg_cred_d_oper_3m_month_1,avg_cred_d_oper_3m_month_2,avg_cred_d_oper_3m_month_3,avg_cred_e_oper_1m_month_1,avg_cred_e_oper_1m_month_2,avg_cred_e_oper_1m_month_3,avg_cred_e_oper_3m_month_1,avg_cred_e_oper_3m_month_2,avg_cred_e_oper_3m_month_3,avg_cred_f_oper_1m_month_1,avg_cred_f_oper_1m_month_2,avg_cred_f_oper_1m_month_3,avg_cred_f_oper_3m_month_1,avg_cred_f_oper_3m_month_2,avg_cred_f_oper_3m_month_3,avg_cred_g_oper_1m_month_1,avg_cred_g_oper_1m_month_2,avg_cred_g_oper_1m_month_3,avg_cred_g_oper_3m_month_1,avg_cred_g_oper_3m_month_2,avg_cred_g_oper_3m_month_3,avg_cred_h_oper_1m_month_1,avg_cred_h_oper_1m_month_2,avg_cred_h_oper_1m_month_3,...,cnt_deb_f_oper_3m_month_3,cnt_deb_g_oper_1m_month_1,cnt_deb_g_oper_1m_month_2,cnt_deb_g_oper_1m_month_3,cnt_deb_g_oper_3m_month_1,cnt_deb_g_oper_3m_month_2,cnt_deb_g_oper_3m_month_3,cnt_deb_h_oper_1m_month_1,cnt_deb_h_oper_1m_month_2,cnt_deb_h_oper_1m_month_3,cnt_deb_h_oper_3m_month_1,cnt_deb_h_oper_3m_month_2,cnt_deb_h_oper_3m_month_3,ft_registration_date_month_1,ft_registration_date_month_2,ft_registration_date_month_3,ogrn_days_end_month_month_1,ogrn_days_end_month_month_2,ogrn_days_end_month_month_3,ogrn_days_end_quarter_month_1,ogrn_days_end_quarter_month_2,ogrn_days_end_quarter_month_3,ogrn_month_month_1,ogrn_month_month_2,ogrn_month_month_3,ogrn_year_month_1,ogrn_year_month_2,ogrn_year_month_3,okved_month_1,okved_month_2,okved_month_3,segment_month_1,segment_month_2,segment_month_3,start_cluster_month_1,start_cluster_month_2,start_cluster_month_3,sum_of_paym_1y_month_1,sum_of_paym_1y_month_2,sum_of_paym_1y_month_3,sum_of_paym_2m_month_1,sum_of_paym_2m_month_2,sum_of_paym_2m_month_3,sum_of_paym_6m_month_1,sum_of_paym_6m_month_2,sum_of_paym_6m_month_3
0,0,-0.4528,-0.4528,-0.4528,-0.9934,-0.9934,-0.9934,-0.0693,-0.0693,-0.0693,-0.0579,-0.0579,-0.0579,-0.1877,-0.1877,-0.1877,-0.1246,-0.1246,-0.1246,-0.2868,-0.2868,-0.2868,-0.2751,-0.2751,-0.2751,0.5789,0.1088,0.1704,0.5045,0.4758,0.3247,-0.1166,-0.1166,-0.1166,-0.1032,-0.1032,-0.1032,0.4415,-0.0957,-0.1334,0.1964,0.1318,0.0355,1.8045,0.6478,0.8779,...,0.2861,0.5139,0.5139,0.5139,0.4581,0.4581,0.4581,0.6541,0.6528,0.6534,0.5536,0.5519,0.5510,2.4522,2.4699,2.4859,-0.4886,-0.4886,-0.4886,-0.1351,-0.1351,-0.1351,ogrn_month_0,ogrn_month_0,ogrn_month_0,ogrn_year_1,ogrn_year_1,ogrn_year_1,okved_30,okved_30,okved_30,segment_1,segment_1,segment_1,"{α, γ}","{α, γ}","{α, γ}",0.5115,0.4864,0.4805,0.9423,0.6457,0.4036,0.5360,0.5364,0.6132
1,1,-0.4528,-0.4528,-0.4528,-0.9934,-0.9934,-0.9934,-0.0693,-0.0693,-0.0693,-0.0579,-0.0579,-0.0579,-0.1877,-0.1877,-0.1877,-0.1246,-0.1246,-0.1246,-0.2868,-0.2868,-0.1764,-0.2751,-0.2751,-0.2387,0.1295,-0.0411,-0.0101,0.2257,0.0689,0.0312,-0.1166,-0.1166,-0.1166,-0.1032,-0.1032,-0.1032,-0.0824,-0.1219,-0.0719,-0.0260,-0.0253,-0.0205,-0.2904,-0.2176,-0.1917,...,0.2861,0.5152,0.5165,0.5152,0.4589,0.4598,0.4598,0.6611,0.6580,0.6617,0.5671,0.5599,0.5608,0.1740,0.1918,0.2078,0.3243,0.3243,0.3243,1.2587,1.2587,1.2587,ogrn_month_5,ogrn_month_5,ogrn_month_5,ogrn_year_15,ogrn_year_15,ogrn_year_15,okved_5,okved_5,okved_5,segment_1,segment_1,segment_1,{other},{other},{other},0.0520,0.0336,0.0395,0.0141,-0.0576,-0.0921,0.0438,0.0350,0.0252
2,2,-0.4528,-0.4528,-0.4528,-0.9934,-0.9934,-0.9934,-0.0693,-0.0693,-0.0693,-0.0579,-0.0579,-0.0579,-0.1877,-0.1877,-0.1877,-0.1246,-0.1246,-0.1246,0.4713,0.3273,0.8310,0.4467,0.4975,0.5481,-0.1559,-0.1575,-0.1516,-0.1716,-0.1710,-0.1703,-0.1166,-0.1166,-0.1166,-0.1032,-0.1032,-0.1032,-0.1334,-0.1334,-0.1334,-0.0303,

In [146]:
df = pd.read_parquet("/kaggle/input/df-restore-cal-avg-start-cluster-3-pqt/df_restore_cal_avg_start_cluster_3_unbalance_karim_2.0") 

In [119]:
# df['start_cluster_month_3'] = df_org1['start_cluster_month_3']

In [147]:
df['start_cluster_month_3'].value_counts()

start_cluster_month_3
{α}          202475
{}            23146
{α, η}        22475
{other}       16686
{α, γ}        16496
{α, β}         5318
{α, δ}         4246
{α, ε}         2804
{α, θ}         2437
{α, ψ}         1491
{α, μ}          776
{α, ε, η}       642
{α, λ}          405
{α, ε, θ}       402
{α, ε, ψ}       174
{λ}              21
{α, π}            6
Name: count, dtype: int64

In [148]:
df

,id,avg_a_oper_1m_month_3,avg_a_oper_3m_month_1,avg_a_oper_3m_month_3,avg_b_oper_1m_month_2,avg_b_oper_1m_month_3,avg_b_oper_3m_month_2,avg_c_oper_1m_month_1,avg_c_oper_1m_month_2,avg_c_oper_1m_month_3,avg_c_oper_3m_month_1,avg_cred_d_oper_1m_month_1,avg_cred_d_oper_1m_month_2,avg_cred_d_oper_1m_month_3,avg_cred_e_oper_1m_month_3,avg_cred_f_oper_1m_month_1,avg_cred_f_oper_1m_month_2,avg_cred_f_oper_1m_month_3,avg_cred_f_oper_3m_month_1,avg_cred_f_oper_3m_month_3,avg_cred_g_oper_1m_month_1,avg_cred_g_oper_1m_month_2,avg_cred_g_oper_1m_month_3,avg_cred_h_oper_1m_month_1,avg_cred_h_oper_1m_month_2,avg_cred_h_oper_1m_month_3,avg_cred_h_oper_3m_month_1,avg_cred_h_oper_3m_month_3,avg_deb_d_oper_1m_month_3,avg_deb_d_oper_3m_month_1,avg_deb_e_oper_3m_month_1,avg_deb_e_oper_3m_month_3,avg_deb_f_oper_1m_month_1,avg_deb_f_oper_1m_month_2,avg_deb_f_oper_1m_month_3,avg_deb_f_oper_3m_month_1,avg_deb_f_oper_3m_month_3,avg_deb_h_oper_1m_month_1,avg_deb_h_oper_1m_month_2,avg_deb_h_oper_1m_month_3,avg_deb_h_oper_3m_month_1,avg_deb_h_oper_3m_month_3,balance_amt_avg_month_1,balance_amt_avg_month_3,channel_code_month_1,channel_code_month_2,...,cnt_days_cred_f_oper_1m_month_2,cnt_days_cred_f_oper_1m_month_3,cnt_days_cred_g_oper_1m_month_1,cnt_days_cred_g_oper_1m_month_3,cnt_days_cred_h_oper_1m_month_1,cnt_days_cred_h_oper_1m_month_3,cnt_days_deb_e_oper_1m_month_1,cnt_days_deb_e_oper_1m_month_2,cnt_days_deb_e_oper_1m_month_3,cnt_days_deb_f_oper_1m_month_1,cnt_days_deb_f_oper_1m_month_2,cnt_days_deb_f_oper_1m_month_3,cnt_days_deb_g_oper_1m_month_1,cnt_days_deb_g_oper_1m_month_3,cnt_days_deb_h_oper_1m_month_1,cnt_days_deb_h_oper_1m_month_3,cnt_deb_d_oper_1m_month_1,cnt_deb_d_oper_1m_month_2,cnt_deb_d_oper_1m_month_3,cnt_deb_e_oper_3m_month_3,cnt_deb_f_oper_3m_month_1,cnt_deb_g_oper_1m_month_1,cnt_deb_g_oper_1m_month_3,cnt_deb_h_oper_3m_month_1,end_cluster_month_3,ft_registration_date_month_3,ogrn_days_end_month_month_1,ogrn_days_end_quarter_month_2,ogrn_month_month_1,ogrn_month_month_2,ogrn_month_month_3,ogrn_year_month_1,ogrn_year_month_2,ogrn_year_month_3,okved_month_1,okved_month_2,okved_month_3,segment_month_1,segment_month_2,segment_month_3,start_cluster_month_1,start_cluster_month_2,start_cluster_month_3,sum_of_paym_2m_month_1,sum_of_paym_2m_month_3,sum_of_paym_6m_month_1
0,0,18,16,16,16,16,14,14,14,15,17,6,6,7,16,7,7,5,8,8,8,10,9,12,15,15,12,12,12,7,13,14,8,17,17,7,7,15,16,16,13,16,17,17,channel_code_5,channel_code_5,...,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,{other},19,19,19,ogrn_month_0,ogrn_month_0,ogrn_month_0,ogrn_year_1,ogrn_year_1,ogrn_year_1,okved_30,okved_30,okved_30,segment_1,segment_1,segment_1,"{α, γ}","{α, γ}","{α, γ}",17,17,17
1,1,18,16,16,16,16,14,14,14,15,17,6,6,7,16,7,7,5,8,8,8,10,9,12,15,15,12,12,12,7,13,14,8,17,17,7,7,15,16,16,13,16,17,17,channel_code_2,channel_code_2,...,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,{other},19,19,19,ogrn_month_5,ogrn_month_5,ogrn_month_5,ogrn_year_15,ogrn_year_15,ogrn_year_15,okved_5,okved_5,okved_5,segment_1,segment_1,segment_1,{other},{other},{other},17,17,17
2,2,18,16,16,16,16,14,14,14,15,17,6,6,7,16,7,7,5,8,8,8,10,9,12,15,15,12,12,12,7,13,14,8,17,17,7,7,15,16,16,13,16,17,17,channel_code_12,channel_code_12,...,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,{α},19,19,19,ogrn_month_9,ogrn_month_9,ogrn_month_9,ogrn_year_19,ogrn_year_19,ogrn_year_19,okved_10,okved_10,okved_10,segment_3,segment_3,segment_3,{α},{α},{α},17,17,17
3,3,0,16,16,16,0,14,0,14,0,17,0,6,0,0,0,7,0,8,8,0,10,0,0,15,0,12,12,0,7,13,14,0,17,0,7,7,0,16,0,13,16,17,17,channel_code_14,channel_code_14,...,19,0,0,0,0,0,0,19,0,0,19,0,0,0,0,0,0,19,0,19,19,0,0,19,"{α, γ}",19,19,19,ogrn_month_4,ogrn_month_4,ogrn_month_4,ogrn_year_10,ogrn_year_10,ogrn_year_10,okved_10,okved_10,okved_10,segment_3,segment_3,segment_3,{α},{α},{α},17,17,17
4,4,18,16,16,16,16,14,14,14,15,17,6,6,7,16,7,7,5,8,8,8,10,9,12,15,15,12,12,12,7,13,14,8,17,17,7,7,15,16,16,13,16,17,17,c

In [149]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Columns: 116 entries, id to sum_of_paym_6m_month_1
dtypes: int64(91), object(25)
memory usage: 265.5+ MB


Обозначение категориальных признаков

In [150]:
cat_names = list(df.select_dtypes(exclude= 'number').columns)

In [151]:
cat_names

['channel_code_month_1',
 'channel_code_month_2',
 'channel_code_month_3',
 'city_month_1',
 'city_month_2',
 'city_month_3',
 'city_type_month_1',
 'city_type_month_2',
 'city_type_month_3',
 'end_cluster_month_3',
 'ogrn_month_month_1',
 'ogrn_month_month_2',
 'ogrn_month_month_3',
 'ogrn_year_month_1',
 'ogrn_year_month_2',
 'ogrn_year_month_3',
 'okved_month_1',
 'okved_month_2',
 'okved_month_3',
 'segment_month_1',
 'segment_month_2',
 'segment_month_3',
 'start_cluster_month_1',
 'start_cluster_month_2',
 'start_cluster_month_3']

In [152]:

# label_encoders = {}

# for col in cat_names:
#     print(col)
#     label_encoders[col] = LabelEncoder().fit(pivot_df[col])

#     train_df[col] = label_encoders[col].transform(train_df[col])
#     test_df[col] = label_encoders[col].transform(test_df[col])

### Создаем выборки для валидации и обучения

Деление на обучающую и тестовую, исправляем дизбаланс классов, если не требуется, пропустить эти блоки до следующего текста

In [153]:
train_df = df[df['id']< 200000]
test_df = df[df['id'] >= 200000]

In [154]:
X = train_df.drop(["id"], axis=1) # оставляю end_cluster чтобы получить пропорцию классов, а потом ниже удалю в коде
y = train_df["end_cluster_month_3"]


In [155]:
x_train, x_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=0.2,
                                                  random_state=42)

In [129]:
x_train['end_cluster_month_3'].value_counts().median()

1432.0

In [360]:
# Определение желаемого количества экземпляров каждого класса
desired_class_count = 40000  # Укажите ваше желаемое количество экземпляров

# Обработка дисбаланса классов
balanced_data = pd.DataFrame()
for cluster in x_train['end_cluster_month_3'].unique():
    cluster_data = x_train[x_train['end_cluster_month_3'] == cluster]
    if len(cluster_data) < desired_class_count:
        resampled_data = resample(
            cluster_data, replace=True, n_samples=desired_class_count, random_state=42)
    else:
        resampled_data = cluster_data.sample(
            n=desired_class_count, replace=False, random_state=42)
    balanced_data = pd.concat([balanced_data, resampled_data])

In [299]:
desired_class_counts = {
    '{α, β}': 3000,        # Желаемое количество экземпляров для класса α, β
    '{α, θ}': 2000,        # Желаемое количество экземпляров для класса α, θ
    '{α, ε}': 2000,        # Желаемое количество экземпляров для класса α, ε
    '{α, δ}': 2000,        # Желаемое количество экземпляров для класса α, δ
    '{α, ψ}': 1000,        # Желаемое количество экземпляров для класса α, ψ
    '{α, μ}': 1000,        # Желаемое количество экземпляров для класса α, μ
    '{α, ε, η}': 1000,     # Желаемое количество экземпляров для класса α, ε, η
    '{α, ε, θ}': 1000,     # Желаемое количество экземпляров для класса α, ε, θ
    '{α, λ}': 1000,        # Желаемое количество экземпляров для класса α, λ
    '{α, ε, ψ}': 500,     # Желаемое количество экземпляров для класса α, ε, ψ
    '{λ}': 500,           # Желаемое количество экземпляров для класса λ
    '{α, π}': 100         # Желаемое количество экземпляров для класса α, π
    # Добавьте другие классы и их желаемое количество экземпляров
}

balanced_data = pd.DataFrame()
for cluster in x_train['end_cluster_month_3'].unique():
    cluster_data = x_train[x_train['end_cluster_month_3'] == cluster]
    desired_count = desired_class_counts.get(cluster)
    if desired_count is None:
        # Если для данного класса нет указанного желаемого количества, оставить данные без изменений
        balanced_data = pd.concat([balanced_data, cluster_data])
    else:
        if len(cluster_data) < desired_count:
            resampled_data = resample(
                cluster_data, replace=True, n_samples=desired_count, random_state=42)
        else:
            resampled_data = cluster_data.sample(
                n=desired_count, replace=False, random_state=42)
        balanced_data = pd.concat([balanced_data, resampled_data])



In [300]:
balanced_data['end_cluster_month_3'].value_counts()

end_cluster_month_3
{α}          84479
{}           33282
{other}      13280
{α, η}       10016
{α, γ}        9022
{α, β}        3000
{α, θ}        2000
{α, δ}        2000
{α, ε}        2000
{α, μ}        1000
{α, ε, θ}     1000
{α, ε, η}     1000
{α, ψ}        1000
{α, λ}        1000
{α, ε, ψ}      500
{λ}            500
{α, π}         100
Name: count, dtype: int64

In [301]:
x_train = balanced_data

In [98]:
# Calculate current class proportions
class_proportions = x_train['end_cluster_month_3'].value_counts(normalize=True)

# Identify underrepresented classes
underrepresented_classes = class_proportions[class_proportions < 0.02].index

# Create an empty list to store additional rows
additional_rows = []

# Calculate the number of additional rows needed for each underrepresented class
for class_label in underrepresented_classes:
    current_proportion = class_proportions[class_label]
    desired_proportion = 0.02
    current_count = (x_train['end_cluster_month_3'] == class_label).sum()
    desired_count = int(desired_proportion * len(train_df))
    additional_count = desired_count - current_count

    if additional_count > 0:
        additional_samples = x_train[x_train['end_cluster_month_3'] == class_label].sample(n=additional_count, replace=True)
        additional_rows.append(additional_samples)

# Concatenate additional rows with the original dataframe
if additional_rows:
    x_train = pd.concat([x_train] + additional_rows, ignore_index=True)

# Verify the class proportions after balancing
print(x_train['end_cluster_month_3'].value_counts())


end_cluster_month_3
{α}          84479
{}           33282
{other}      13280
{α, η}       10016
{α, γ}        9022
{α, β}        4000
{α, θ}        4000
{α, δ}        4000
{α, μ}        4000
{α, ε}        4000
{α, ε, θ}     4000
{α, ε, η}     4000
{α, ψ}        4000
{α, ε, ψ}     4000
{λ}           4000
{α, λ}        4000
{α, π}        4000
Name: count, dtype: int64


In [99]:
x_train['end_cluster_month_3'].value_counts()

end_cluster_month_3
{α}          84479
{}           33282
{other}      13280
{α, η}       10016
{α, γ}        9022
{α, β}        4000
{α, θ}        4000
{α, δ}        4000
{α, μ}        4000
{α, ε}        4000
{α, ε, θ}     4000
{α, ε, η}     4000
{α, ψ}        4000
{α, ε, ψ}     4000
{λ}           4000
{α, λ}        4000
{α, π}        4000
Name: count, dtype: int64

In [303]:
y = X['end_cluster_month_3']
X = X.drop(['end_cluster_month_3'], axis=1)
x_val = x_val.drop(['end_cluster_month_3'], axis=1)

In [156]:
y_train = x_train['end_cluster_month_3']
x_train = x_train.drop(['end_cluster_month_3'], axis=1)
x_val = x_val.drop(['end_cluster_month_3'], axis=1)

In [157]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape

((160000, 114), (160000,), (40000, 114), (40000,))

In [158]:
X.shape, y.shape, x_val.shape, y_val.shape

((200000, 115), (200000,), (40000, 114), (40000,))

In [159]:
X['end_cluster_month_3'].value_counts()

end_cluster_month_3
{α}          105884
{}            40770
{other}       16661
{α, η}        13066
{α, γ}        11136
{α, β}         3706
{α, θ}         2170
{α, ε}         1819
{α, δ}         1767
{α, ψ}          926
{α, μ}          716
{α, ε, η}       548
{α, ε, θ}       373
{α, λ}          293
{α, ε, ψ}       111
{λ}              49
{α, π}            5
Name: count, dtype: int64

Деление на тестовую и обучающую выборку без дизбаланса

In [104]:
# X = train_df.drop(["id", "date", "end_cluster"], axis=1)
# y = train_df["end_cluster"]

# x_train, x_val, y_train, y_val = train_test_split(X, y,
#                                                   test_size=0.2,
#                                                   random_state=42)

## Обучение модели

Кросс-валидация для CatBoost (не нажимать если не разбираешься). Если не разбираешься, идти к следующему блоку текста

In [134]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200000 entries, 0 to 199999
Columns: 115 entries, avg_a_oper_1m_month_3 to sum_of_paym_6m_month_1
dtypes: int64(90), object(25)
memory usage: 177.0+ MB


In [135]:
cat_names = X.columns[1:]

In [136]:
cat_names

Index(['avg_a_oper_3m_month_1', 'avg_a_oper_3m_month_3',
       'avg_b_oper_1m_month_2', 'avg_b_oper_1m_month_3',
       'avg_b_oper_3m_month_2', 'avg_c_oper_1m_month_1',
       'avg_c_oper_1m_month_2', 'avg_c_oper_1m_month_3',
       'avg_c_oper_3m_month_1', 'avg_cred_d_oper_1m_month_1',
       ...
       'okved_month_3', 'segment_month_1', 'segment_month_2',
       'segment_month_3', 'start_cluster_month_1', 'start_cluster_month_2',
       'start_cluster_month_3', 'sum_of_paym_2m_month_1',
       'sum_of_paym_2m_month_3', 'sum_of_paym_6m_month_1'],
      dtype='object', length=114)

In [108]:
def weighted_roc_auc(y_true, y_pred, labels, weights_dict):
    unnorm_weights = np.array([weights_dict[label] for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()
    classes_roc_auc = roc_auc_score(y_true, y_pred, labels=labels,
                                    multi_class="ovr", average=None)
    return sum(weights * classes_roc_auc)

In [109]:
cluster_weights = pd.read_excel("/kaggle/input/alfa-hak1/cluster_weights.xlsx").set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()

In [136]:
import optuna
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Define your data x_train, x_val, y_train, y_val
# Replace them with your actual data
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 50, 2000, step=50),
        'depth': trial.suggest_int('depth', 4, 10, step=1),
        'learning_rate': trial.suggest_categorical('learning_rate', np.linspace(0.01, 0.1, 10)),
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 0, 10),
        'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian']),  # Removed 'MVS'
        'one_hot_max_size': trial.suggest_int('one_hot_max_size', 2, 16),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations', 1, 20),
        'max_ctr_complexity': trial.suggest_int('max_ctr_complexity', 1, 10),
        'leaf_estimation_method': trial.suggest_categorical('leaf_estimation_method', ['Newton', 'Gradient']),
        'random_strength': trial.suggest_float('random_strength', 0.1, 1.0, step=0.1)
    }
    
    if params['bootstrap_type'] == 'Bayesian':
        # Remove 'subsample' parameter if 'bootstrap_type' is 'Bayesian'
        params.pop('subsample', None)
    
    catboost = CatBoostClassifier(loss_function='MultiClass', early_stopping_rounds=20, random_seed=42, task_type="GPU", **params)
    
    catboost.fit(
        x_train, y_train,
        eval_set=(x_val, y_val),
        verbose=500  # выводить статус через каждые 500 итераций
    )
    
    y_pred = catboost.predict(x_val)
    return accuracy_score(y_val, y_pred)


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

best_params = study.best_params
best_score = study.best_value

print("Best parameters found: ", best_params)
print("Best score: ", best_score)


[I 2024-04-14 02:11:41,177] A new study created in memory with name: no-name-10750355-881c-4fc0-adf2-1fdddcb6aa32


0:	learn: 2.6828363	test: 2.6748748	best: 2.6748748 (0)	total: 309ms	remaining: 2m 18s
449:	learn: 0.6725129	test: 0.7844276	best: 0.7844276 (449)	total: 1m 43s	remaining: 0us
bestTest = 0.7844276367
bestIteration = 449


[I 2024-04-14 02:13:27,177] Trial 0 finished with value: 0.74445 and parameters: {'iterations': 450, 'depth': 10, 'learning_rate': 0.05000000000000001, 'l2_leaf_reg': 1, 'bootstrap_type': 'Bayesian', 'one_hot_max_size': 2, 'leaf_estimation_iterations': 20, 'max_ctr_complexity': 5, 'leaf_estimation_method': 'Gradient', 'random_strength': 0.7000000000000001}. Best is trial 0 with value: 0.74445.


0:	learn: 2.7754848	test: 2.7721453	best: 2.7721453 (0)	total: 163ms	remaining: 8s
49:	learn: 1.3220449	test: 1.2384202	best: 1.2384202 (49)	total: 8.36s	remaining: 0us
bestTest = 1.238420215
bestIteration = 49


[I 2024-04-14 02:13:37,166] Trial 1 finished with value: 0.735025 and parameters: {'iterations': 50, 'depth': 10, 'learning_rate': 0.020000000000000004, 'l2_leaf_reg': 6, 'bootstrap_type': 'Bayesian', 'one_hot_max_size': 9, 'leaf_estimation_iterations': 16, 'max_ctr_complexity': 10, 'leaf_estimation_method': 'Gradient', 'random_strength': 0.6}. Best is trial 0 with value: 0.74445.


...

0:	learn: 2.7795016	test: 2.7765844	best: 2.7765844 (0)	total: 62.8ms	remaining: 1m 30s
500:	learn: 0.9096807	test: 0.8264630	best: 0.8264630 (500)	total: 17.8s	remaining: 33.8s
1000:	learn: 0.8707643	test: 0.7991041	best: 0.7991041 (1000)	total: 34.7s	remaining: 15.6s
1449:	learn: 0.8525463	test: 0.7904142	best: 0.7904142 (1449)	total: 49.9s	remaining: 0us
bestTest = 0.7904141602
bestIteration = 1449


[I 2024-04-14 04:57:16,466] Trial 98 finished with value: 0.7445 and parameters: {'iterations': 1450, 'depth': 6, 'learning_rate': 0.020000000000000004, 'l2_leaf_reg': 4, 'bootstrap_type': 'Bayesian', 'one_hot_max_size': 14, 'leaf_estimation_iterations': 14, 'max_ctr_complexity': 2, 'leaf_estimation_method': 'Gradient', 'random_strength': 0.1}. Best is trial 93 with value: 0.74705.


0:	learn: 2.6163305	test: 2.6106252	best: 2.6106252 (0)	total: 50.9ms	remaining: 1m 29s
500:	learn: 0.7849216	test: 0.7750863	best: 0.7750863 (500)	total: 43.5s	remaining: 1m 48s
bestTest = 0.7717205078
bestIteration = 800
Shrink model to first 801 iterations.


[I 2024-04-14 04:58:30,018] Trial 99 finished with value: 0.745575 and parameters: {'iterations': 1750, 'depth': 7, 'learning_rate': 0.04000000000000001, 'l2_leaf_reg': 5, 'bootstrap_type': 'Bayesian', 'one_hot_max_size': 15, 'leaf_estimation_iterations': 11, 'max_ctr_complexity': 8, 'leaf_estimation_method': 'Newton', 'random_strength': 0.7000000000000001}. Best is trial 93 with value: 0.74705.


Best parameters found:  {'iterations': 1950, 'depth': 7, 'learning_rate': 0.06000000000000001, 'l2_leaf_reg': 9, 'bootstrap_type': 'Bayesian', 'one_hot_max_size': 14, 'leaf_estimation_iterations': 11, 'max_ctr_complexity': 1, 'leaf_estimation_method': 'Gradient', 'random_strength': 0.2}
Best score:  0.74705


In [ ]:
def objective(trial):
    data, target = load_breast_cancer(return_X_y=True)
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.3)
    
    learning_rate=np.linspace(0.01,0.1,10)
max_depth=np.arange(2, 18, 2)
colsample_bylevel=np.arange(0.3, 0.8, 0.1)
iterations=np.arange(50, 1000, 50)
l2_leaf_reg=np.arange(0,10)
bagging_temperature=np.arange(0,100,10)
    param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "3gb",
    }


    gbm = cb.CatBoostClassifier(**param)

    gbm.fit(x_train, y_train, eval_set=[(x_val, y_val)], verbose=15, early_stopping_rounds=100, task_type="GPU", loss_function='MultiClass')

    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    accuracy = accuracy_score(valid_y, pred_labels)
    return accuracy

iterations=2024,
                           depth=8,
                           learning_rate=0.075,
                           random_seed=47,
                           loss_function='MultiClass',
                           task_type="GPU",
                           devices='0',
                           early_stopping_rounds=20

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=600)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
from catboost import cv

params = {
    'loss_function': 'MultiClass',
    'iterations': 512,
    'random_seed': 43,
    'learning_rate': 0.1,
    'depth': 12,
    'task_type': 'GPU',
    'devices':'0'
}

cv_data = cv(
    params=params,
    pool=pool1,
    fold_count=5, # Разбивка выборки на 5 кусочков
    shuffle=True, # Перемешаем наши данные
    partition_random_seed=0,
    plot=True, # Никуда без визуализатора
    stratified=True,
    verbose=30
)

In [ ]:
cv_data

In [ ]:
best_value = np.min(cv_data['test-Logloss-mean'])
best_iter = np.argmin(cv_data['test-Logloss-mean'])
print("Best validation Logloss score, stratified: {:.4f}+/-{:.3f} on step {}".format(
best_value, cv_data['test-Logloss-std'][best_iter], best_iter))

## Начало обучения

### CatBoostClassifier

In [ ]:
from catboost import CatBoostClassifier
import numpy as np

def train_catboost(model, x_train, y_train, x_val, y_val, cat_names):
    model.fit(
        x_train, y_train,
        cat_features=np.array(cat_names),
        eval_set=(x_val, y_val),
        verbose=100
    )
    model.save_model('catboost_model_start_claster.json')
    feature_importance = model.get_feature_importance(prettified=True)
    return feature_importance

# Пример использования функции
model = CatBoostClassifier(iterations=2028,
                           depth=8,
                           learning_rate=0.075,
                           random_seed=47,
                           loss_function='MultiClass',
                           task_type="GPU",
                           devices='0',
                           early_stopping_rounds=20
                           )

feature_names = x_train.columns.tolist()  # Предположим, что у вас есть DataFrame x_train с данными
selector = RecursiveFeatureElimination(
    variables=feature_names, estimator=model, scoring='accuracy', threshold=0.05)
selector.fit(x_train, y_train)

model #13, classes balanced

In [160]:
model = CatBoostClassifier(iterations=2000,
                           depth=6,
                           learning_rate=0.075,
                           random_seed=47,
                           loss_function='MultiClass',
                           task_type="GPU",
                           devices='0',
                           early_stopping_rounds=20
                          )


In [161]:
params = {
    'iterations': 1950,
    'depth': 7,
    'learning_rate': 0.075,
    'l2_leaf_reg': 7,
    'bootstrap_type': 'Bayesian',
    'one_hot_max_size': 14,
    'leaf_estimation_iterations': 11,
    'max_ctr_complexity': 1,
    'leaf_estimation_method': 'Gradient',
    'random_strength': 0.2
}

# # Создадим модель с указанными параметрами
# model = CatBoostClassifier(**params)


model = CatBoostClassifier(random_seed=47,
                           loss_function='MultiClass',
                           task_type="GPU",
                           devices='0',
                           early_stopping_rounds=20,
                          **params)

In [162]:
def train_catboost(model, x_train, y_train, x_val, y_val, cat_names):

    model.fit(
    x_train, y_train,
    cat_features=np.array(cat_names),
    eval_set=(x_val, y_val),
    verbose=15 # через сколько итераций выводить стату
    )
    model.save_model('catboost_model_no_end_claster.json') # сохранение модели
    feature_importance = model.get_feature_importance(prettified=True) # датасет с важностью признаков

    return feature_importance

In [163]:
y_val

119737        {α}
72272     {other}
158154        {α}
65426     {other}
30074         {α}
           ...   
4174           {}
91537          {}
156449        {α}
184376     {α, η}
6584       {α, γ}
Name: end_cluster_month_3, Length: 40000, dtype: object

In [164]:
cat_names = x_train.columns[1:]

In [165]:
feature_importance = train_catboost(model, x_train, y_train, x_val, y_val, cat_names)

0:	learn: 2.6362227	test: 2.6250785	best: 2.6250785 (0)	total: 70.4ms	remaining: 2m 17s
15:	learn: 1.2778809	test: 1.1868322	best: 1.1868322 (15)	total: 792ms	remaining: 1m 35s
30:	learn: 1.0492799	test: 0.9493107	best: 0.9493107 (30)	total: 1.54s	remaining: 1m 35s
45:	learn: 0.9849770	test: 0.8886371	best: 0.8886371 (45)	total: 2.33s	remaining: 1m 36s
60:	learn: 0.9560354	test: 0.8634347	best: 0.8634347 (60)	total: 3.08s	remaining: 1m 35s
75:	learn: 0.9392835	test: 0.8493418	best: 0.8493418 (75)	total: 3.84s	remaining: 1m 34s
90:	learn: 0.9276638	test: 0.8411825	best: 0.8411825 (90)	total: 4.61s	remaining: 1m 34s
105:	learn: 0.9192515	test: 0.8346699	best: 0.8346699 (105)	total: 5.38s	remaining: 1m 33s
120:	learn: 0.9128107	test: 0.8302620	best: 0.8302620 (120)	total: 6.12s	remaining: 1m 32s
135:	learn: 0.9073626	test: 0.8263784	best: 0.8263784 (135)	total: 6.87s	remaining: 1m 31s
150:	learn: 0.9017314	test: 0.8228593	best: 0.8228593 (150)	total: 7.63s	remaining: 1m 30s
165:	learn: 0.

In [166]:
feature_importance

,Feature Id,Importances
0,start_cluster_month_3,38.6926
1,start_cluster_month_1,6.9883
2,okved_month_3,4.3574
3,start_cluster_month_2,3.6887
4,okved_month_1,3.4984
...,...,...
109,cnt_deb_d_oper_1m_month_3,0.0000
110,cnt_deb_e_oper_3m_month_3,0.0000
111,cnt_deb_f_oper_3m_month_1,0.0000
112,cnt_deb_g_oper_1m_month_3,0.0000


Модель LGBM, блоки кода не нажимать чтобы не сбросилась model CatBoost. Я еще не тестил эту модель, переходи к метрикам

### LGBM

В качестве базовой модели возьмем LGBM обучим на всех признаках

In [ ]:
model = LGBMClassifier(verbosity=-1, random_state=42, n_jobs=-1)
model.fit(x_train, y_train)

LGBMClassifier(n_jobs=-1, random_state=42, verbosity=-1)

### Ансамбль 2

In [20]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split


In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Определите категориальные и числовые признаки
categorical_features  = X.select_dtypes(exclude = 'number').columns
numerical_features = X.select_dtypes(include = 'number').columns

# Создайте конвейеры для предобработки категориальных и числовых признаков
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),  
    ('encoder', OneHotEncoder())  
])

numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Заполните пропущенные значения средним значением
    ('scaler', StandardScaler())  # Масштабируйте числовые признаки
])

# Создайте объект ColumnTransformer для объединения конвейеров предобработки
preprocessor = ColumnTransformer([
    ('cat', categorical_pipeline, categorical_features),
    ('num', numerical_pipeline, numerical_features)
])

base_models = [
    ('catboost', CatBoostClassifier(iterations=1024,
                                     depth=7,
                                     learning_rate=0.075,
                                     random_seed=43,
                                     loss_function='MultiClass',
                                     task_type="GPU",
                                     devices='0',
                                     early_stopping_rounds=20,
                                     verbose=500)),
    ('lgbm', LGBMClassifier(n_estimators=100, max_depth=3, verbosity=-1, random_state=42, n_jobs=-1, tree_method= 'gpu_hist')),
    ('extra_trees', ExtraTreesClassifier(n_estimators=50, max_depth=3, random_state=42),
    ('xgboost', XGBClassifier(n_estimators=100, max_depth=3, verbosity=0, random_state=42, n_jobs=-1, tree_method= 'gpu_hist')))
]

meta_model = RandomForestClassifier(n_estimators=50, random_state=42, max_depth=5)


preprocessor = ColumnTransformer([
    ('cat', categorical_pipeline, categorical_features),
    ('num', numerical_pipeline, numerical_features)
])

# Объедините предобработку и модель в один конвейер
model = Pipeline([
    ('preprocessor', preprocessor),
    ('stacking_classifier', StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5))
])

model.fit(X_train, y_train)


# Оцените качество модели
accuracy = model.score(X_test, y_test)
print("Accuracy:", accuracy)

In [ ]:
model.save_model('stack_model_no_end_claster.json')

In [42]:
%%time
base_models = [
    ('catboost', CatBoostClassifier(iterations=1024,
                                     depth=7,
                                     learning_rate=0.075,
                                     random_seed=43,
                                    cat_features=np.array(cat_names),
                                     loss_function='MultiClass',
                                     task_type="GPU",
                                     devices='0',
                                     early_stopping_rounds=20,
                                     verbose=500)),
    ('lgbm', LGBMClassifier(n_estimators=100, max_depth=3, verbosity=-1, random_state=42, n_jobs=-1, tree_method= 'gpu_hist')),
    ('extra_trees', ExtraTreesClassifier(n_estimators=50, max_depth=3, random_state=42),
    ('xgboost', XGBClassifier(n_estimators=100, max_depth=3, verbosity=0, random_state=42, n_jobs=-1, tree_method= 'gpu_hist')))
]

meta_model = RandomForestClassifier(n_estimators=50, random_state=42, max_depth=5)

model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)

model.fit(X, y)

### Метрики

Зададим функцию для взвешенной метрики roc auc

In [167]:
def weighted_roc_auc(y_true, y_pred, labels, weights_dict):
    unnorm_weights = np.array([weights_dict[label] for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()
    classes_roc_auc = roc_auc_score(y_true, y_pred, labels=labels,
                                    multi_class="ovr", average=None)
    return sum(weights * classes_roc_auc)

In [168]:
cluster_weights = pd.read_excel("/kaggle/input/alfa-hak1/cluster_weights.xlsx").set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()

Проверка работы модели

In [169]:
y_pred_proba = model.predict_proba(x_val)
y_pred_proba.shape

(40000, 17)

In [170]:
y_val

119737        {α}
72272     {other}
158154        {α}
65426     {other}
30074         {α}
           ...   
4174           {}
91537          {}
156449        {α}
184376     {α, η}
6584       {α, γ}
Name: end_cluster_month_3, Length: 40000, dtype: object

In [171]:
y_pred_proba

array([[9.73895279e-03, 1.49316873e-02, 3.14106553e-02, ...,
        1.21980947e-03, 8.96249692e-01, 6.47088076e-04],
       [1.53277158e-01, 9.67763943e-03, 1.91494048e-02, ...,
        1.93054554e-03, 5.07137438e-01, 1.12678603e-03],
       [1.58187409e-02, 2.41756168e-02, 1.93307567e-02, ...,
        1.55123362e-03, 8.73605879e-01, 8.78606640e-04],
       ...,
       [9.19966523e-02, 7.21991861e-02, 2.53635292e-03, ...,
        2.44924066e-03, 2.52599945e-01, 8.31185861e-04],
       [1.07943824e-01, 2.00053202e-02, 1.36584702e-03, ...,
        8.81049084e-04, 1.06689774e-01, 5.36683912e-04],
       [8.66038380e-02, 1.58751854e-02, 6.75301289e-03, ...,
        1.79239805e-03, 2.48573923e-01, 7.61769736e-04]])

In [172]:
 model.classes_

array(['{other}', '{}', '{α, β}', '{α, γ}', '{α, δ}', '{α, ε, η}',
       '{α, ε, θ}', '{α, ε, ψ}', '{α, ε}', '{α, η}', '{α, θ}', '{α, λ}',
       '{α, μ}', '{α, π}', '{α, ψ}', '{α}', '{λ}'], dtype=object)

In [173]:
weighted_roc_auc(y_val, y_pred_proba, model.classes_, weights_dict)

0.8850643745606058

## Прогноз на тестовой выборке

In [54]:
test_df

,id,avg_a_oper_1m_month_3,avg_a_oper_3m_month_1,avg_a_oper_3m_month_3,avg_b_oper_1m_month_2,avg_b_oper_1m_month_3,avg_b_oper_3m_month_2,avg_c_oper_1m_month_1,avg_c_oper_1m_month_2,avg_c_oper_1m_month_3,avg_c_oper_3m_month_1,avg_cred_d_oper_1m_month_1,avg_cred_d_oper_1m_month_2,avg_cred_d_oper_1m_month_3,avg_cred_e_oper_1m_month_3,avg_cred_f_oper_1m_month_1,avg_cred_f_oper_1m_month_2,avg_cred_f_oper_1m_month_3,avg_cred_f_oper_3m_month_1,avg_cred_f_oper_3m_month_3,avg_cred_g_oper_1m_month_1,avg_cred_g_oper_1m_month_2,avg_cred_g_oper_1m_month_3,avg_cred_h_oper_1m_month_1,avg_cred_h_oper_1m_month_2,avg_cred_h_oper_1m_month_3,avg_cred_h_oper_3m_month_1,avg_cred_h_oper_3m_month_3,avg_deb_d_oper_1m_month_3,avg_deb_d_oper_3m_month_1,avg_deb_e_oper_3m_month_1,avg_deb_e_oper_3m_month_3,avg_deb_f_oper_1m_month_1,avg_deb_f_oper_1m_month_2,avg_deb_f_oper_1m_month_3,avg_deb_f_oper_3m_month_1,avg_deb_f_oper_3m_month_3,avg_deb_h_oper_1m_month_1,avg_deb_h_oper_1m_month_2,avg_deb_h_oper_1m_month_3,avg_deb_h_oper_3m_month_1,avg_deb_h_oper_3m_month_3,balance_amt_avg_month_1,balance_amt_avg_month_3,channel_code_month_1,channel_code_month_2,...,cnt_days_cred_f_oper_1m_month_2,cnt_days_cred_f_oper_1m_month_3,cnt_days_cred_g_oper_1m_month_1,cnt_days_cred_g_oper_1m_month_3,cnt_days_cred_h_oper_1m_month_1,cnt_days_cred_h_oper_1m_month_3,cnt_days_deb_e_oper_1m_month_1,cnt_days_deb_e_oper_1m_month_2,cnt_days_deb_e_oper_1m_month_3,cnt_days_deb_f_oper_1m_month_1,cnt_days_deb_f_oper_1m_month_2,cnt_days_deb_f_oper_1m_month_3,cnt_days_deb_g_oper_1m_month_1,cnt_days_deb_g_oper_1m_month_3,cnt_days_deb_h_oper_1m_month_1,cnt_days_deb_h_oper_1m_month_3,cnt_deb_d_oper_1m_month_1,cnt_deb_d_oper_1m_month_2,cnt_deb_d_oper_1m_month_3,cnt_deb_e_oper_3m_month_3,cnt_deb_f_oper_3m_month_1,cnt_deb_g_oper_1m_month_1,cnt_deb_g_oper_1m_month_3,cnt_deb_h_oper_3m_month_1,end_cluster_month_3,ft_registration_date_month_3,ogrn_days_end_month_month_1,ogrn_days_end_quarter_month_2,ogrn_month_month_1,ogrn_month_month_2,ogrn_month_month_3,ogrn_year_month_1,ogrn_year_month_2,ogrn_year_month_3,okved_month_1,okved_month_2,okved_month_3,segment_month_1,segment_month_2,segment_month_3,start_cluster_month_1,start_cluster_month_2,start_cluster_month_3,sum_of_paym_2m_month_1,sum_of_paym_2m_month_3,sum_of_paym_6m_month_1
200000,200000,18,16,16,16,16,14,14,14,15,17,6,6,7,16,7,7,5,8,8,8,10,9,12,15,15,12,12,12,7,13,14,8,17,17,7,7,15,16,16,13,16,17,17,channel_code_12,channel_code_12,...,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,missing,19,19,19,ogrn_month_8,ogrn_month_8,ogrn_month_8,ogrn_year_11,ogrn_year_11,ogrn_year_11,okved_0,okved_0,okved_0,segment_3,segment_3,segment_3,{α},{α},{α},17,17,17
200001,200001,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17,17,channel_code_9,channel_code_9,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,missing,19,19,19,ogrn_month_10,ogrn_month_10,ogrn_month_10,ogrn_year_11,ogrn_year_11,ogrn_year_11,okved_6,okved_6,okved_6,segment_3,segment_3,segment_3,{α},{α},{α},0,0,0
200002,200002,18,16,17,16,16,14,14,14,15,17,6,6,7,16,7,7,5,8,8,8,10,9,12,15,15,12,12,12,7,13,14,8,17,17,7,7,15,16,16,13,16,17,17,channel_code_12,channel_code_12,...,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,missing,19,19,19,ogrn_month_6,ogrn_month_6,ogrn_month_6,ogrn_year_11,ogrn_year_11,ogrn_year_11,okved_63,okved_63,okved_63,segment_3,segment_3,segment_3,{other},{other},"{α, μ}",17,17,17
200003,200003,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17,17,channel_code_7,channel_code_7,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,missing,19,19,19,ogrn_month_9,ogrn_month_9,ogrn_month_9,ogrn_year_4,ogrn_year_4,ogrn_year_4,okved_5,okved_5,okved_5,segment_3,segment_3,segment_3,{α},{α},{α},0,0,0
200004,200004,18,0,16,0,16,0,0,0,15,0,0,0,7,16,0,0,5,0,8,0,0,9,0,0,15,0,12,12,0,0,14,0,0,17,0,7,0,0,16,0,16,0,17,missing,channel_code_7,...,0,19,0,19,0,19,0,0,19,0,0,19,0,19,

In [174]:
sample_submission_df = pd.read_csv("/kaggle/input/alfa-hak1/sample_submission.csv") # поменять на свой
last_m_test_df = test_df
last_m_test_df = last_m_test_df.drop(["id" , 'end_cluster_month_3'], axis=1)

pool2 = Pool(data=last_m_test_df, cat_features=np.array(cat_names))

test_pred_proba = model.predict_proba(pool2) # last_m_test_df
test_pred_proba_df = pd.DataFrame(test_pred_proba, columns=model.classes_)
sorted_classes = sorted(test_pred_proba_df.columns.to_list())
test_pred_proba_df = test_pred_proba_df[sorted_classes]

sample_submission_df[sorted_classes] = test_pred_proba_df
sample_submission_df.to_csv("catboost_submission_akm_super_shit.csv", index=False) # сохранение модели

In [ ]:
sample_submission_df = pd.read_csv("/content/drive/MyDrive/Alfa-hak/sample_submission.csv") # поменять на свой
last_m_test_df = test_df
last_m_test_df = last_m_test_df.drop("id", axis=1)

pool2 = Pool(data=last_m_test_df, cat_features=np.array(cat_names))

test_pred_proba = model.predict_proba(pool2) # last_m_test_df
test_pred_proba_df = pd.DataFrame(test_pred_proba, columns=model.classes_)
sorted_classes = sorted(test_pred_proba_df.columns.to_list())
test_pred_proba_df = test_pred_proba_df[sorted_classes]

sample_submission_df[sorted_classes] = test_pred_proba_df
sample_submission_df.to_csv("catboost_submission_akm_ba.csv", index=False) # сохранение модели

In [ ]:
sample_submission_df.shape # если (100000, 18), то все правильно. Если нет - надо писать Кариму и спрашивать(

In [ ]:
sample_submission_df

In [368]:
sample_submission_df

,id,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ}
0,200000,0.0188,0.0075,0.0122,0.0324,0.0154,0.0014,0.0020,0.0010,0.0142,0.0056,0.0101,0.0036,0.0046,0.0006,0.0053,0.8649,0.0005
1,200001,0.0141,0.5673,0.0021,0.0033,0.0016,0.0012,0.0018,0.0011,0.0040,0.0159,0.0031,0.0017,0.0021,0.0005,0.0019,0.3753,0.0029
2,200002,0.6788,0.0038,0.0063,0.0731,0.0100,0.0039,0.0050,0.0280,0.0181,0.0277,0.0138,0.0084,0.0057,0.0008,0.0616,0.0543,0.0008
3,200003,0.0282,0.5678,0.0017,0.0030,0.0014,0.0010,0.0015,0.0008,0.0024,0.0142,0.0026,0.0012,0.0017,0.0004,0.0015,0.3694,0.0011
4,200004,0.0860,0.1002,0.0184,0.0594,0.0133,0.0089,0.0047,0.0018,0.0131,0.0755,0.0099,0.0036,0.0137,0.0011,0.0092,0.5769,0.0043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,299995,0.0092,0.4733,0.0023,0.0039,0.0014,0.0005,0.0013,0.0006,0.0020,0.0020,0.0019,0.0017,0.0009,0.0004,0.0012,0.4958,0.0017
99996,299996,0.0271,0.0631,0.0166,0.0454,0.0116,0.0032,0.0042,0.0020,0.0154,0.0142,0.0111,0.0098,0.0072,0.0012,0.0046,0.7603,0.0032
99997,299997,0.0230,0.0251,0.0172,0.0469,0.0188,0.0016,0.0041,0.0024,0.0192,0.0072,0.0123,0.0052,0.0072,0.0012,0.0068,0.8009,0.0009
99998,299998,0.1614,0.1143,0.0216,0.0509,0.0091,0.0073,0.0073,0.0028,0.0177,0.1076,0.0148,0.0126,0.0103,0.0015,0.0165,0.4339,0.0105


In [371]:
sample_submission_df['{α, ψ}'].value_counts().sort_index()

{α, ψ}
0.0001    1
0.0001    1
0.0001    1
0.0001    1
0.0001    1
         ..
0.8508    1
0.8517    1
0.8556    1
0.8573    1
0.8635    1
Name: count, Length: 99972, dtype: int64